In [ ]:
!pip install -q gradio

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 206 kB 58.9 MB/s 
     |████████████████████████████████| 1.9 MB 45.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 961 kB 62.7 MB/s 
     |████████████████████████████████| 3.0 MB 44.5 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import PIL.Image as Image
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
tf_model_url = 'https://tfhub.dev/google/on_device_vision/classifier/landmarks_classifier_asia_V1/1'
label_map_url = 'https://www.gstatic.com/aihub/tfhub/labelmaps/landmarks_classifier_asia_V1_label_map.csv'
image_shape= (321, 321)

In [ ]:
# import layers from model url
classifier = tf.keras.Sequential([hub.KerasLayer(tf_model_url, 
                                                 input_shape= image_shape + (3,),                  #can only concatenate tuple with tuple
                                                 output_key= 'predictions:logits')])

In [ ]:
df = pd.read_csv(label_map_url)

In [ ]:
df.head(3)

,id,name
0,0,Bagha Shahi Mosque
1,1,City Palace
2,2,The Immaculate Conception Cathedral of Cubao


In [ ]:
# dict mapping as keys are the ids and values are the monuments
label_map = dict(zip(df.id, df.name))
label_map

In [ ]:
img_loc= '/content/IMG_20180104_170608.jpg'
img = Image.open(img_loc).resize(image_shape)
img

In [ ]:
# intensity values have to be in 0-1 (model requirement)
img = np.array(img) / 255.0
img.shape

(321, 321, 3)

In [ ]:
# one prediction at a time : add new axis at the start
img = img[np.newaxis, ...]
img.shape

(1, 321, 321, 3)

In [ ]:
# prediction
result = classifier.predict(img)
result                                                 #shows list of probablility values

array([[ 0.11734014,  0.06656237,  0.21670969, ...,  0.21727484,
        -0.04035402,  0.10864833]], dtype=float32)

In [ ]:
# model has been trained on 98960 classes
result.shape

(1, 98960)

In [ ]:
# show name of the max probability value monument
label_map[np.argmax(result)]

'Petronas Twin Towers'

## WebApp: Gradio

In [ ]:
class_names=list(label_map.values())

In [ ]:
# backend code for webapp
def classify_image(image):
    img = np.array(image)/255.0
    img = img[np.newaxis, ...]
    prediction = classifier.predict(img)
    return label_map[np.argmax(prediction)]

In [ ]:
# web interface
image = gr.inputs.Image(shape=(321, 321))
label = gr.outputs.Label(num_top_classes=3)

gr.Interface(
    classify_image, 
    image, 
    label,
    capture_session=True).launch(debug=True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://19638.gradio.app
Interface loading below...
